In [1]:
#! pip install git+https://github.com/martinetd/samloader.git

In [2]:
#from samloader import versionfetch, fusclient, request, crypt
import xml.etree.ElementTree as ET
from tqdm import tqdm
import base64
import sys
import os
from math import floor
import json
import os
import csv

#def checkupdate(model, csc):
#  return versionfetch.getlatestver(model, csc)

class DownloadException(Exception):
  pass

# def get_updates_data(client, fw, model, region):
#   req = request.binaryinform(fw, model, region, client.nonce)
#   #print(req)
#   resp = client.makereq("NF_DownloadBinaryInform.do", req)
#   root = ET.fromstring(resp)
#   status = int(root.find("./FUSBody/Results/Status").text)
#   if status != 200:
#     print(resp)
#     print("DownloadBinaryInform returned {}, firmware for {}-{} ({}) could not be found?".format(status, model, region, fw))
#     raise DownloadException
#   factory_do_exist = bool(int(root.find("./FUSBody/Put/FACTORY_DO_EXIST/Data").text))
#   last_modified = root.find("./FUSBody/Put/LAST_MODIFIED/Data").text
#   return (factory_do_exist, last_modified)

In [3]:
# Model names got from https://en.wikipedia.org/wiki/Samsung_Galaxy and then
# checked the actual name using https://www.sammobile.com/firmwares/archive/

models = [
 # 2024
  "SM-S921B",
  "SM-S926B",
  "SM-S928B",
  "SM-A556B",
  "SM-A356B",
  "SM-E156B",
  "SM-M556B",
  "SM-M156S",
  "SM-M356B",
  "SM-A065F",
  "SM-A166B",
 # 2023
  "SM-M146B",
  "SM-A346N",
  "SM-A546B",
  "SM-M546B",
  "SM-E146B",
  "SM-A245F",
  "SM-F731B",
  "SM-F946B",
  "SM-A055F",
  "SM-S711B",
  "SM-A256B",
  "SM-A156B",
  "SM-S918N",
  "SM-S916N",
  "SM-A146U",
  "SM-S911U",
 # 2022
  "SM-F936B",
  "SM-F721B",
  "SM-G736B",
  "SM-E135F",
  "SM-M135F",
  "SM-M536B",
  "SM-A736B",
  "SM-A736B",
  "SM-A536B",
  "SM-A336B",
  "SM-A336B",
  "SM-M336B",
  "SM-M236B",
  "SM-E236B",
  "SM-A236B",
  "SM-A235F",
  "SM-A135F",
  "SM-X906B",
  "SM-X900",
  "SM-X806B",
  "SM-X800",
  "SM-X706B",
  "SM-X706B",
  "SM-X700",
  "SM-S908B",
  "SM-S906B",
  "SM-S901B",
  "SM-G990B",
# 2021 updated
  "SM-A136B",
  "SM-X200",
  "SM-X205",
  "SM-A035F",
  "SM-A032F",
# 2021
  "SM-A426B",
  "SM-E426B",
  "SM-M526B",
  "SM-A528B",
  "SM-F926B",
  "SM-F711B",
  "SM-E225F",
  "SM-M325F",
  "SM-A226B",
  "SM-A225F",
  "SM-A225M",
  "SM-T730",
  "SM-T736B",
  "SM-T220",
  "SM-T225",
  "SM-E5260",
  "SM-M426B",
  "SM-E025F",
  "SM-F127G",
  "SM-A725F",
  "SM-A526B",
  "SM-A525F",
  "SM-A325F",
  "SM-M625F",
  "SM-E625F",
  "SM-M127G",
  "SM-M022G",
  "SM-A022G",
  "SM-G991B",
  "SM-G996B",
  "SM-G998B",
  "SM-A326B",
  "SM-M025F",
# 2020
  "SM-A025G",
  "SM-A125F",
  "SM-A125F",
  "SM-A426B",
  "SM-F415F",
  "SM-G780F",
  "SM-G780G",
  "SM-M515F",
  "SM-N980F",
  "SM-N985F",
  "SM-F707B",
  "SM-T870",
  "SM-M317F",
  "SM-M017F",
  "SM-M015G",
  "SM-A217M",
  "SM-A716B",
  "SM-A516B",
  "SM-A215U",
  "SM-P610",
  "SM-G980F",
  "SM-G985F",
  "SM-G988B",
  "SM-M115F",
  "SM-A315F",
  "SM-A415F",
  "SM-M215F",
  "SM-A115F",
  "SM-M315F",
  "SM-F700F",
  "SM-G715FN",
  "SM-N770F",
  "SM-G770F",
# 2019
  "SM-A015F",
  "SM-A715F",
  "SM-A515F",
  "SM-M307F",
  "SM-A207F",
  "SM-M107F",
  "SM-A707F",
  "SM-A507FN",
  "SM-A307G",
  "SM-A908B",
  "SM-F900F",
  "SM-A107F",
  "SM-A102U",
  "SM-N970N",
  "SM-N975F",
  "SM-M405F",
  "SM-G977B",
  "SM-A6060",
  "SM-A805F",
  "SM-A705F",
  "SM-A405F",
  "SM-A205F",
  "SM-A202F",
  "SM-A260F",
  "SM-G975N",
  "SM-G973N",
  "SM-G970F",
  "SM-A505F",
  "SM-A305N",
  "SM-A105F",
  "SM-T720",
  "SM-T510",
  "SM-M305F",
  "SM-M105F",
  "SM-M205F",
]


cscs = [
  "EUX",
  "XAA",
  "EVR",
  "3IE",
  "BRI",
  "KOO",
  "INS",
  "CAU",
  "GTO",
  "ZTO",
  "CHC",
  "XSG",
  "SER",
  "ROM",
  "SPR",
  "TGY",
  "DSA",
  "USC",
  "TMB",
]


In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

#client = fusclient.FUSClient()

def get_val(div):
  return div.get_text().split(" : ")[1]

def get_builds(model, csc):
  url = f"https://doc.samsungmobile.com/{model}/{csc}/doc.html"
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  sel = soup.find("select", id="sel_lang")
  opt = [s['value'] for s in sel.find_all("option") if s['value'].endswith("kor.html")][0]
  url = urljoin(url, opt)
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  rows = soup.find_all("div", class_="row")
  builds = []
  for row in rows:
    if row.get_text().strip() != '':
      divs = row.find_all("div")
      data = {"build_number": get_val(divs[0]), "android_version": get_val(divs[1]), "release": get_val(divs[2]), "patch": get_val(divs[3])}
      builds.append(data)
  return builds

In [5]:
import traceback

# client = fusclient.FUSClient()

devices = dict()
for model in models:
  for csc in cscs:
  #  try:
  #    version = checkupdate(model, csc)
  #    (exists, last_modified) = get_updates_data(client, version, model, csc)
  #    devices[f"{model}-{csc}"] = {"exists": exists, "last_modified": last_modified, "version": version}
  #    if not exists:
  #      print(f"WARNING: model {model}-{csc} does not exist (?)")
  #    print(f"found {model}-{csc}...")
  #  except DownloadException:
  #    devices[f"{model}-{csc}"] = {"exists": True, "last_modified": None, "version": version}
  #    print(f"found {model}-{csc}...")
  #  except:
  #    print(f"error for {model}-{csc}")
  #    traceback.print_exc()
  #    continue
    try:
      builds = get_builds(model, csc)
      print(f"found builds for {model}-{csc}...")
      devices[f"{model}-{csc}"] = {}
      devices[f"{model}-{csc}"]["builds"] = builds
    except Exception as e:
      pass
      #print(f"error for {model}-{csc}")
      #traceback.print_exc()

found builds for SM-S921B-EUX...


/usr/lib/python3.11/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


found builds for SM-S921B-EVR...
found builds for SM-S921B-INS...
found builds for SM-S921B-CAU...
found builds for SM-S921B-GTO...
found builds for SM-S921B-ZTO...
found builds for SM-S921B-XSG...
found builds for SM-S921B-SER...
found builds for SM-S926B-EUX...
found builds for SM-S926B-EVR...
found builds for SM-S926B-INS...
found builds for SM-S926B-CAU...
found builds for SM-S926B-GTO...
found builds for SM-S926B-ZTO...
found builds for SM-S926B-XSG...
found builds for SM-S926B-SER...
found builds for SM-S928B-EUX...
found builds for SM-S928B-EVR...
found builds for SM-S928B-INS...
found builds for SM-S928B-CAU...
found builds for SM-S928B-GTO...
found builds for SM-S928B-ZTO...
found builds for SM-S928B-XSG...
found builds for SM-S928B-SER...
found builds for SM-A556B-EUX...
found builds for SM-A556B-EVR...
found builds for SM-A356B-EUX...
found builds for SM-A356B-EVR...
found builds for SM-E156B-INS...
found builds for SM-M556B-INS...
found builds for SM-M556B-ZTO...
found buil

In [6]:
# create a dict that contains all known builds
def merge_builds(dev):
  builds = dict()
  for id in dev:
    d = dev[id]
    if "builds" in d and d["builds"]:
      for b in d["builds"]:
        builds[b["build_number"]] = b
  return builds

def get_build_number(version):
  return version.split("/")[0]

In [7]:
from dateutil.parser import parse
import datetime

# check if there are devices with no builds. Additionally, if according to the
# FUS server there is an update that is not reported on the device webpage,
# check if it was reported on another device webpage, and if so add the data
# to the list of builds
def mark_missing_devices(devices):
  known_builds = merge_builds(devices)
  for id in devices:
    d = devices[id]
    d["missing_data"] = False
    if "builds" in d:
      build_dates = [parse(b["release"]) for b in d["builds"]]
      if build_dates:
        last_build = max(d["builds"], key=lambda b : parse(b["release"]))
        if d["last_modified"] is None:
          last_modified = None
        else:
          dt = parse(d["last_modified"])
          last_modified =  datetime.datetime(dt.year, dt.month, dt.day)
        if not d["version"].startswith(last_build["build_number"]):
            ver = get_build_number(d["version"])
            if ver in known_builds:
              print(f"adding build {ver} to {id}...")
              b = known_builds[ver]
              new_build = {**b}
              if last_modified:
                new_build["release"] = last_modified.strftime("%Y-%m-%d")
              d["builds"].insert(0, new_build)
            else:
              print(f"marking {id} as missing data [{ver}]...")
              d["missing_data"] = True
      else:
        print(f"no build dates for {id}")
        d["missing_data"] = True
    else:
      print(f"no builds for {id}")
      d["missing_data"] = True

In [9]:
def filter_builds(builds, date="2025-01-01"):
  return [b for b in builds if b["release"] < date]

for id in devices:
  d = devices[id]
  if("builds" in d):
    d["builds"] = filter_builds(d["builds"])

In [10]:
output_dir = "drive/My Drive/cves"
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, "patches-2024.json")

with open(output_file, "w") as f:
  json.dump(devices, f)

In [11]:
def export_csv(devices, filename):
  with open(filename, "w", newline='') as f:
    header = ["id", "model", "csc", "patch", "release_date", "build"]
    writer = csv.writer(f)
    writer.writerow(header)
    for id in devices:
      d = devices[id]
      # if not d["missing_data"]:
      if True:
        model, csc = id.rsplit("-", 1)
        for b in d["builds"]:
          writer.writerow([id, model, csc, b["patch"], b["release"], b["build_number"]])


In [12]:
output_csv = os.path.join(output_dir, "patches-valid-2024.csv")

export_csv(devices, output_csv)

In [13]:
import pandas as pd
import os

output_dir = "drive/My Drive/cves"
output_csv = os.path.join(output_dir, "patches-valid-2024.csv")
df = pd.read_csv(output_csv)
df

,id,model,csc,patch,release_date,build
0,SM-S921B-EUX,SM-S921B,EUX,2024-12-01,2024-12-11,S921BXXS5AXKA
1,SM-S921B-EUX,SM-S921B,EUX,2024-11-01,2024-12-02,S921BXXU5AXK4
2,SM-S921B-EUX,SM-S921B,EUX,2024-11-01,2024-11-04,S921BXXS5AXJ8
3,SM-S921B-EUX,SM-S921B,EUX,2024-10-01,2024-10-10,S921BXXS4AXI8
4,SM-S921B-EUX,SM-S921B,EUX,2024-09-01,2024-09-25,S921BXXS3AXHD
...,...,...,...,...,...,...
17798,SM-M205F-XSG,SM-M205F,XSG,2019-08-01,2019-10-04,M205FDDU2BSH3
17799,SM-M205F-XSG,SM-M205F,XSG,2019-05-01,2019-07-22,M205FDDU1BSF4
17800,SM-M205F-XSG,SM-M205F,XSG,2019-05-01,2019-06-22,M205FDDU1BSF1
17801,SM-M205F-XSG,SM-M205F,XSG,2019-03-01,2019-04-27,M205FDDU1ASD3


In [14]:
print(len(models))
print(len(cscs))

161
19


In [15]:
len(df["id"].unique())

651